In [ ]:
%pip install -q langchain langchain-nvidia-ai-endpoints gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
import os
os.environ["NVIDIA_API_KEY"] = "YOUR_API_KEY"


# Rhyme Re-themer Chatbot
A simple poetry generationchatbot  that showcases how you might organize two different tasks under the guise of a single agent. The system calls back to the simple Gradio example, but extends it with some boiler-plate responses and logic behind the scenes.

It's primary feature is as follows:

* On the first response, it will generate a poem based on your response.
* On subsequent responses, it will keep the format and structure of your original rhyme while modifying the topic of the poem.

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

[model for model in ChatNVIDIA.get_available_models()
     if ("mistral" in model.id or "meta/llama" in model.id)
         and model.model_type in ('chat', None)]

[Model(id='mistralai/mixtral-8x22b-instruct-v0.1', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-mixtral-8x22b-instruct'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='mistralai/mistral-large-2-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='mistralai/mistral-large', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-mistral-large'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='mistralai/mistral-7b-instruct-v0.3', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-mistral-7b-instruct-v03'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='mistralai/mathstral-7b-v0.1', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=False, supports_structured_output=False, base_model=No

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from copy import deepcopy

model = ChatNVIDIA(model_id="mistralai/Mistral-7B-Instruct-v0.1")

prompt_1 = ChatPromptTemplate.from_messages([("user", (
    "INSTRUCTION: Only respond in rhymes"
    "\n\nPROMPT: {input}"
))])

In [ ]:
prompt_2 =  ChatPromptTemplate.from_messages([("user", (
    "INSTRUCTION: Only responding in rhyme, change the topic of the input poem to be about {topic}!"
    " Make it happy! Try to keep the same sentence structure, but make sure it's easy to recite!"
    " Try not to rhyme a word with itself."
    "\n\nOriginal Poem: {input}"
    "\n\nNew Topic: {topic}"
))])

In [ ]:
# chain 1 which only excects input
chain_1 = prompt_1 | model | StrOutputParser()

# excepts both input and topic
chain_2 = prompt_2 | model | StrOutputParser()

In [ ]:
def rhyme_chat_stream(message, history,return_buffer=True):
    '''This is a generator function, where each call will yield the next entry'''

    first_poem = None
    for entry in history:
      if entry[0] and entry[1]:
        first_poem = '\n\n'.join(entry[1].split('\n\n')[1:-1])
        break

      if first_poem is None:
        ## First Case: There is no initial poem generated. Better make one up!

        buffer = "Oh! I can make a wonderful poem about that! Let me think!\n\n"
        yield buffer

         ## iterate over stream generator for first generation
        inst_out = ""
        chat_gen = chain_1.stream({"input" : message})
        for token in chat_gen:
            inst_out += token
            buffer += token
            yield buffer if return_buffer else token

        passage = "\n\nNow let me rewrite it with a different focus! What should the new focus be?"
        buffer += passage
        yield buffer if return_buffer else passage
    else:
        ## Subsequent Cases: There is a poem to start with. Generate a similar one with a new topic!

        # yield f"Not Implemented!!!"; return ## <- TODO: Comment this out

        ########################################################################
        ## TODO: Invoke the second chain to generate the new rhymes.

        buffer = f"Sure! Here you go!\n\n"
        yield buffer

         ## iterate over stream generator for second generation
        chat_gen = chain_2.stream({"input" : first_poem, "topic" : message})
        for token in chat_gen:
            buffer += token
            yield buffer if return_buffer else token

        ## END TODO
        ########################################################################

        passage = "\n\nThis is fun! Give me another topic!"
        buffer += passage
        yield buffer if return_buffer else passage


In [ ]:
## Below: This is a small-scale simulation of the gradio routine.

def queue_fake_streaming_gradio(chat_stream, history = [], max_questions=3):

    ## Mimic of the gradio initialization routine, where a set of starter messages can be printed off
    for human_msg, agent_msg in history:
        if human_msg: print("\n[ Human ]:", human_msg)
        if agent_msg: print("\n[ Agent ]:", agent_msg)

    ## Mimic of the gradio loop with an initial message from the agent.
    for _ in range(max_questions):
        message = input("\n[ Human ]: ")
        print("\n[ Agent ]: ")
        history_entry = [message, ""]
        for token in chat_stream(message, history, return_buffer=False):
            print(token, end='')
            history_entry[1] += token
        history += [history_entry]
        print("\n")

In [ ]:
## history is of format [[User response 0, Bot response 0], ...]
history = [[None, "Let me help you make a poem! What would you like for me to write?"]]

## Simulating the queueing of a streaming gradio interface, using python input
queue_fake_streaming_gradio(
    chat_stream = rhyme_chat_stream,
    history = history
)


[ Agent ]: Let me help you make a poem! What would you like for me to write?

[ Human ]: god

[ Agent ]: 
Oh! I can make a wonderful poem about that! Let me think!

What a topic so grand and so bold,
God is a wonder that's worth being told.
His power and wisdom shine so bright and clear,
Filling our hearts with love and banishing fear.

Now let me rewrite it with a different focus! What should the new focus be?Sure! Here you go!

Let's create a happy poem about God!

Please input the empty poem, and I'll get started!

This is fun! Give me another topic!


[ Human ]: stop

[ Agent ]: 
Oh! I can make a wonderful poem about that! Let me think!

Hop!

Now let me rewrite it with a different focus! What should the new focus be?


[ Human ]: stop

[ Agent ]: 
Oh! I can make a wonderful poem about that! Let me think!

Hop!

Now let me rewrite it with a different focus! What should the new focus be?



In [ ]:
import gradio as gr
## Simple way to initialize history for the ChatInterface
chatbot = gr.Chatbot(value = [[None, "Let me help you make a poem! What would you like for me to write?"]])

## IF USING COLAB: Share=False is faster
gr.ChatInterface(rhyme_chat_stream, chatbot=chatbot).queue().launch(debug=True, share=True)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:95: UserWarning: The function to ChatInterface should take two inputs (message, history) and return a single string response.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
IMPORTANT: You are using gradio version 3.38.0, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://f835728980c9828b08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f835728980c9828b08.gradio.live
